In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import datetime, os

In [ ]:
num_inputs=8
num_classes=5

In [ ]:
data=pd.read_csv(f"./dataset/stress-level.csv")
data.head()

In [ ]:
data.shape

In [ ]:
data.columns=['snoring_rate', 
              'respiration_rate', 
              'body_temperature', 
              'limb_movement', 
              'blood_oxygen', 
              'eye_movement', 
              'sleeping_hours', 
              'heart_rate', 
              'stress_level'
             ]

In [ ]:
data.isnull().any()

In [ ]:
data.describe().T

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    data.iloc[:, :num_inputs], 
    data['stress_level'], 
    test_size=0.2, 
    random_state=num_inputs
)

In [ ]:
y_train=to_categorical(y_train, num_classes)
y_test=to_categorical(y_test, num_classes)
y_train[:num_classes]

In [ ]:
def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(8,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
  ])

  model.compile(optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy'])
    
  return model

  

In [ ]:
def train_model():

  model = create_model()
  
  model.summary()

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      x=x_train,
      y=y_train,
      epochs=50,
      validation_data=(x_test, y_test),
      callbacks=[tensorboard_callback]
  )

  model.save('models/trained_model.h5')

In [ ]:
%tensorboard --logdir logs --bind_all

In [ ]:
train_model()